In [ ]:
import pandas as pd
from sodapy import Socrata
import warnings
warnings.simplefilter('ignore')
# Display Setting
from IPython.display import display
pd.options.display.max_colwidth=100
pd.options.display.float_format="{:.2f}".format
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly as pl
import chart_studio.plotly as py
import cufflinks as cf
from plotly.offline import download_plotlyjs,init_notebook_mode,plot,iplot
from sklearn.metrics import auc,roc_curve,classification_report,confusion_matrix,mean_absolute_error,mean_squared_error,root_mean_squared_error
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.svm import SVC
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import nltk
import string
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
%matplotlib inline

In [9]:
results_df

,source_date,animal_id,type,source_name,ispreviouslyspayedneutered,sex,primary_breed,primary_color,secondary_color,date_of_birth,found_address,name_at_intake,intake_health_condition
0,2025-09-18T15:29:00.000,15067,Dog,Stray,False,Female,Coonhound,Red,White,2020-09-18T00:00:00.000,"903 Cactus Drive, ROUND ROCK, TX, 78681, Unite...",NaN,NaN
1,2025-09-18T15:27:00.000,15615,Dog,ACO - Impound,False,Male,Rhodesian Ridgeback,Red,White,2023-09-18T00:00:00.000,"TX, United States",Bravi,NaN
2,2025-09-18T15:17:00.000,15614,Cat,Community Animal - Program,False,Unknown,Domestic Medium Hair,White,NaN,2023-09-18T00:00:00.000,"7511 Carver Avenue, AUSTIN, TX, 78752, United ...",NaN,NaN
3,2025-09-18T14:51:00.000,15613,Cat,ACO - Impound,False,Female,Domestic Medium Hair,Black,White,2015-09-18T00:00:00.000,"2409 Santa Rita Street, AUSTIN, TX, 78702, Uni...",Delila,NaN
4,2025-09-18T14:45:00.000,15612,Cat,Community Animal - Program,False,Unknown,Domestic Shorthair,Brown Tabby,White,2021-09-18T00:00:00.000,"2100 Deer Run Drive, AUSTIN, TX, 78744, United...",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5386,2025-05-05T10:40:00.000,6885,Dog,Stray,False,Female,German Shepherd,Buff,Cream,2025-01-05T00:00:00.000,"2008 Rinker Ranch Drive, AUSTIN, TX, 78725, Un...",NaN,NaN
5387,2025-05-05T10:25:00.000,6883,Dog,Stray,True,Female,American Pit Bull Terrier,White,Brown,2023-05-05T00:00:00.000,"6200 Loyola Lane, AUSTIN, TX, 78724, United St...",Viola,NaN
5388,2025-05-05T09:55:00.000,6881,Cat,Owner Surrender,False,Female,Himalayan,Cream,Brown,2020-05-05T00:00:00.000,"2200 Willow Creek Drive, AUSTIN, TX, 78741, Un...",Honeysuckle,NaN
5389,2025-05-05T09:05:00.000,6877,Dog,Stray,True,Male,Poodle - Standard,White,White,2012-05-05T00:00:00.000,"4401 Tilley Street, AUSTIN, TX, 78723, United ...",Lucky,NaN
